In [1]:
# load modules
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [2]:
# personal mapbox token
mapbox_access_token = "pk.eyJ1IjoiaHl2aCIsImEiOiJja3RrNWlzcGgwMDQzMm5vODYwcHJvcmRlIn0.QIsGMFNngsjk4GZD_Nou8g"

In [92]:
df = pd.read_csv("final_daft_listings.csv")

### Remove all listings without Latitude/Longitude

In [93]:
df[["Longitude", "Latitude"]].isnull().sum()

Longitude    1033
Latitude     1033
dtype: int64

In [94]:
df.columns

Index(['Address', 'Room', 'Bath', 'Property_type', 'Bedrooms_available',
       'Available_from', 'Available_for', 'Sharing_with', 'Owner_occupied',
       'Preferences', 'Date_entered/renewed', 'Property_views', 'Latitude',
       'Longitude', 'Pre_processed_desc', 'Desc_length', 'Price_per_month',
       'County', 'adjective_counts'],
      dtype='object')

In [95]:
df = df.dropna(subset = ["Longitude", "Latitude"]) # remove all listings that do not have Latitude or Longitude
df = df.dropna(subset = ["County"]) # remove all listing that do not have a County (for checklist)

# resetting indexes
df = df.reset_index(drop=True)

### creating a column that maps random colors to a room type

In [96]:
import random

In [155]:
rooms = list(df["Room"].unique()) # all room types

color_palette = dict.fromkeys(rooms) # create dict with room types as keys

In [165]:
# randomly assigns color hex code to each room type
for i in rooms:
    hexadecimal = ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])] # random hex code
    color_palette.update({i:hexadecimal[0]}) # get the first item of the list

In [166]:
color_palette

{'Double Room': '#3A8655',
 'Single Room': '#ED1AF9',
 'Twin Room': '#AE0C8E',
 'Shared Room': '#D7582B',
 'Single & Double Room': '#A56FC1',
 'Single & Twin Room': '#3F3317',
 'Single & Shared Room': '#949E99',
 'Single, Double & Shared Room': '#B34DD2',
 'Single, Double & Twin Room': '#999DF2',
 'Double & Shared Room': '#5E261A',
 'Double & Twin Room': '#A5A1A7'}

In [167]:
color_palette["Double Room"]

'#3A8655'

In [168]:
# color based on room type 
df["Color_room"] = df["Room"].map(color_palette)

# manually adjusting some colors


# Mapbox

In [255]:
app = dash.Dash(__name__)

app.layout = html.Div([


    html.Div([
        html.Div([
            # checklist for Counties 
            html.Label(children='Counties:'),
            dcc.Checklist(id='sort_by_counties', options=[{"label": str(b), "value": b} for b in sorted(df["County"].unique())],
                          value = [b for b in sorted(df["County"].unique())], inline = True),
            
        # html.Div([
        #     dcc.Dropdown(
        #     id='sort_by_counties',
        #     options=[{"label": str(b), "value": b} for b in sorted(df["County"].unique())],
        #     value = [b for b in sorted(df["County"].unique())] ,style={'width': '50%', 'display': 'inline-block'}
        #     ),

            # Map Legend
            html.Div([
                html.Label(children = "Legend: ", ), # title
                
                html.Li("Double Room", className='circle', style={'background': color_palette["Double Room"],'color':'white','list-style':'inside','text-indent': '30px'}),
                html.Li("Single Room", className= "circle", style = {"background": color_palette["Single Room"], "color":"white", "list-style": "inside", "text-indent": "30px"}),
                html.Li("Twin Room", className = "circle", style = {"background": color_palette["Twin Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Shared Room", className = "circle", style = {"background": color_palette["Shared Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Single & Double Room", className = "circle", style = {"background": color_palette["Single & Double Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Single & Twin Room", className = "circle", style = {"background": color_palette["Single & Twin Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Single & Shared Room", className = "circle", style = {"background": color_palette["Single & Shared Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Single, Double & Shared Room", className = "circle", style = {"background": color_palette["Single, Double & Shared Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Single, Double & Twin Room", className = "circle", style = {"background": color_palette["Single, Double & Twin Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Double & Shared Room", className = "circle", style = {"background": color_palette["Double & Shared Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
                html.Li("Double & Twin Room", className = "circle", style = {"background": color_palette["Double & Twin Room"], "color": "white", "list-style": "inside", "text-indent":"30px"}),
            ]),

        ]),

        # map
        html.Div([
            dcc.Graph(id='graph', config={'displayModeBar': False, 'scrollZoom': True},
                      style={'background': '#00FC87', 'padding-bottom': '2px', 'padding-left': '2px', 'height': '100vh'}
                        )
        ])
    ])
                      
])

# Output of Graph
@app.callback(Output('graph', 'figure'),
              Input('sort_by_counties', 'value'))

def update_figure(chosen_county):
    df_sub = df[(df['County'].isin(chosen_county))]

    #hovertext
    #df_sub["Informationen"][df_sub["Informationen" == None]] = "Keine Informationen vorhanden"

    locations = [go.Scattermapbox(
        lon= list(df_sub["Longitude"]),
        lat= list(df_sub["Latitude"]),
        mode="markers+text",
        marker = {"size": 10,'color' : df_sub['Color_room']},
        unselected={"marker": {'opacity': 1}},
        selected={'marker': {'opacity': 0.5, 'size': 40}},
        textposition='top right',
        hoverinfo="text",
        hovertext=
            ["<b>" + str(df_sub["Address"][i]) + "</b>" + "<br>" +
            "Monthly rent :" + " " + str(df_sub['Price_per_month'][i]) + '<br>' +  
            "Bath : " + " " + str(df_sub["Bath"][i]) + "<br>" +
            "Property type : " + " " + str(df_sub["Property_type"][i]) + "<br>" +
            "Bedrooms available: " + " " + str(df_sub["Bedrooms_available"][i]) + "<br>" +
            "Available from: " + " " + str(df_sub["Available_from"][i]) + "<br>" +
            "Available for: " + " " + str(df_sub["Available_for"][i]) + "<br>" +
            "Sharing with: " + " " + str(df_sub["Sharing_with"][i]) + "<br>" +
            "Owner occupied: " + " " + str(df_sub["Owner_occupied"][i]) + "<br>" +
            "Preferences: " + " " + str(df_sub["Preferences"][i]) + "<br>" +
            "Date entered/renewed: " + " " + str(df_sub["Date_entered/renewed"][i]) + "<br>" +
            "Property views: " + " " + str(df_sub["Property_views"][i])
            for i in range(df_sub.shape[0])],

    )]

    # Return figure
    return {
        ## scattermapbox has two sections: "data" and "layout", the data section already contains the figure
        'data': locations,
        'layout': go.Layout(
            #mapbox_style = "white-bg",
            uirevision='foo',
            # preserves state of figure/map after callback activated
            clickmode= 'event+select',
            hovermode='closest',
            ## default = closest
            hoverdistance=2,
            title=dict(text="Rooms on daft.ie", font=dict(size=30)),
            mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=0,
                #style='carto-darkmatter',
                style = "basic",
                center = dict(
                    lat = 53.350140,
                    lon = -6.266155
                ),
            pitch=40,
            zoom=11.5
            ),
        )
    }

In [ ]:
# start map
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2023 14:56:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 14:56:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 14:56:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 14:56:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 14:56:43] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 14:56:43] "POST /_dash-update-component HTTP/1.1" 200 -
[2023-04-20 14:56:46,096] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\hyvhu\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 108, in pandas._libs.index.IndexEngine.get_l